# Giant Mess - Dealing with Timezones and Daylight Saving Time in Python
## - I went to sleep at 2 am on November 1... - Um, which one?
<img src='images/time.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@andrey-grushnikov-223358?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Andrey Grushnikov</a>
        on 
        <a href='https://www.pexels.com/photo/black-and-white-photo-of-clocks-707676/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Time Zones

The first mechanical clock was invented in the 17th century. During those times, everyone set their clock so that noon was directly overhead and this was not a big problem because everyone traveled on foot and horses. Later, telegraphs and trains were invented and you could suddenly travel around and communicate with people hundreds of miles away in a significantly short amount of time. This led to people realizing they had to have a standard time so, in the 1900s, the greatest leaders and thinkers from developed countries gathered around and divided the world into 24 time zones. 

Greenwich, London got be 0 o'clock because they standardized their time first and the countries west to the UK set their clocks earlier than London and the countries to the West set later. This standardization is also knows as UTC (Universal Coordinate Time). In other words, if you Google 'UTC time now', it will show the time in London.

Today, data moves around the world in the blink of an eye and if date and time information is not timezone aware, it will create a whole host of problems for programmers. In this post, we will talk about how to address such problems, including Daylight Saving Time.

### Setup

To start working with timezones, let's import the necessary modules from `datetime`:

> Before moving on, I highly recommend to read my [previous](https://towardsdatascience.com/date-and-time-objects-in-python-everything-you-need-to-know-10aa3bf121be?source=your_stories_page-------------------------------------) article on datetimes and timedeltas if you don't know already.

In [2]:
from datetime import datetime, timedelta, timezone

We will need `datetime` to work with DateTime objects, `timedelta` to work with time durations and `timezone` to create time differences between UTC standard time. To practice, I will load the bi

In [34]:
# Store times for the moment
london = datetime(2020, 11, 25, 4, 28, 45)
washington = datetime(2020, 11, 24, 20, 28, 45)
print("    Time in London: " + london.isoformat() + '\n')
print("Time in Washington: " + washington.isoformat())

    Time in London: 2020-11-25T04:28:45

Time in Washington: 2020-11-24T20:28:45


As Python documentation says, `london` and `washington` are naive meaning they do not know their timezone. Quick reminder for the notation of timezone info for datetimes:

```
YYYY-MM-DD HH:MM:SS + UTC Offset
```

UTC offset basically means the time difference between UTC and the time zone. For example, Washington is 8 hours behind the UTC, London time, so it has a UTC offset of -8 hours.

In [16]:
# Create an 8 hour difference
offset_dc = timezone(timedelta(hours=-8))
# Insert it into washington
washington = datetime(2020, 11, 24, 20, 28, 45, tzinfo=offset_dc)
print(washington)

2020-11-24 20:28:45-08:00


To create a `timezone` object, we use `timezone` function which accepts a `timedelta` which explains how to translate your time into UTC.

In [17]:
print(type(offset_dc))

<class 'datetime.timezone'>


If you pass this object to any `datetime` under the `tzinfo` argument, it will add the timezone information without changing the time itself, like above.

Now, let's also get the time in India and add a timezone info to compare it with UTC:

In [19]:
# Store indian standard time with +5 hour 30 minute difference
ist = timezone(timedelta(hours=5, minutes=30))
india = datetime(2020, 11, 25, 9, 28, 45, tzinfo=ist)
print(india)

2020-11-25 09:28:45+05:30


How about `london`? Even though we know it is in UTC time, we should indicate its timezone too:

In [22]:
lst = timezone(timedelta(hours=0))
london = datetime(2020, 11, 25, 4, 28, 45, tzinfo=lst)
print(london)

2020-11-25 04:28:45+00:00


Let's print out all three dates for comparison:

In [26]:
print(f'Time in Washington: {washington.isoformat()}')
print(f'Time in London:     {london.isoformat()}')
print(f'Time in Kolkata:    {india.isoformat()}')

Time in Washington: 2020-11-24T20:28:45-08:00
    Time in London: 2020-11-25T04:28:45+00:00
   Time in Kolkata: 2020-11-25T09:28:45+05:30


You should keep in mind that `-8:00` or `+5:30` does not mean that you have to subtract 8 hours or add 5 hours and 30 minutes to get UTC. They just indicate the time difference so if you want to know UTC time from a timezone info, you would take the opposite of the timezone sign and add it to the time.

In the above examples, we had to know the UTC offsets beforehand to insert them into dates. But in practice, you may deal with thousands of locations and you can't possibly know all of their timezones. Besides, timezones always change as governments update their laws regarding time.

Thankfully, the global timezone database IANA, records all historical and future changes to timezones and it is said the database gets updated every 3-4 months. This database is also used by programmers across many languages and you can get it for Python using `dateutil` (built-in) package:

In [28]:
from dateutil import tz

To create a timezone using the information provided in `tz`, we will use `gettz` function. It accepts a string in format of `Continent/Major_City`. For example, creating timezone for Washington DC:

In [33]:
# Get the timezone for America/Vancouver
dc_tz = tz.gettz('America/Vancouver')
# Store time in washington without the time zone info
washington = datetime(2020, 11, 24, 20, 28, 45)
# Add the time zone
dc_changed = washington.replace(tzinfo=dc_tz)
print(dc_changed)

2020-11-24 20:28:45-08:00


To get 